### Import Statements

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Input
from sklearn.metrics import mean_absolute_error, mean_squared_error

### Load the dataset

In [3]:
train = pd.read_csv('../data/train_processed.csv', low_memory=False, index_col='Date').drop(columns=["Date.1"])
    
test = pd.read_csv('../data/test_processed.csv', low_memory=False, index_col='Id').drop(columns=["Date"])